In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
import os
from gpt_model import OpenAIEmbeddingClient

openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAIEmbeddingClient(api_key=openai_api_key)

In [11]:
from utils import load_data_halu_eval, get_cosine_similarity

output_halu_eval_file_path = '../data/output/interrogate_llm_zeroshot/halu_eval_long_answer_output.json'
output_halu_eval_cosine_similarity_file_path = '../data/output/interrogate_llm_zeroshot/halu_eval_long_answer_cosine_similarity.json'

output = load_data_halu_eval(output_halu_eval_file_path)
print(f"Length of HaluEval output: {len(output)}\n")

Length of HaluEval output: 300



In [9]:
from tqdm import tqdm

result = []

for idx, sample in enumerate(tqdm(output, desc="Processing output")):
    question = sample['question']
    reconstruct_questions = sample['reconstruct_questions']

    embedded_question = client.get_embedding(question)
    embedded_reconstruct_question = [client.get_embedding(reconstruct_question) for reconstruct_question in reconstruct_questions]

    cosine_similarity = get_cosine_similarity(embedded_question, embedded_reconstruct_question)
    result.append({
            "id": sample['id'],
            "knowledge": sample["knowledge"],
            "ground_truth": sample["ground_truth"],
            "question": sample["question"],
            "answer": sample["answer"],
            "reconstruct_question": sample['reconstruct_questions'],
            "is_hallucinated": sample['is_hallucinated'],
            "cosine_similarity": float(cosine_similarity)
        })

Processing output: 100%|██████████| 300/300 [19:57<00:00,  3.99s/it]


In [14]:
import json

with open(output_halu_eval_cosine_similarity_file_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=4)